In [1]:
import json
import torch
import pdb
import pyarrow.feather as feather
from typing import List, Tuple, Any, Dict
from PIL import Image
import numpy as np
import pandas as pd
import pickle

from pathlib import Path
from av2.datasets.sensor.av2_sensor_dataloader import AV2SensorDataLoader
from av2.datasets.sensor.sensor_dataloader import SensorDataloader
from av2.datasets.sensor.constants import RingCameras, AnnotationCategories
from av2.geometry.camera.pinhole_camera import PinholeCamera
from av2.map.map_api import ArgoverseStaticMap, DrivableAreaMapLayer
from av2.geometry.se3 import SE3
from av2.geometry.geometry import compute_interior_points_mask
from av2.rendering.rasterize import xyz_to_bev
from av2.structures.cuboid import CuboidList
from av2.utils.io import read_city_SE3_ego, read_img
import random


STATIC = [
    AnnotationCategories.BOLLARD ,
    AnnotationCategories.CONSTRUCTION_BARREL ,
    AnnotationCategories.CONSTRUCTION_CONE ,
    AnnotationCategories.MOBILE_PEDESTRIAN_CROSSING_SIGN ,
    AnnotationCategories.SIGN ,
    AnnotationCategories.STOP_SIGN ,
    AnnotationCategories.TRAFFIC_LIGHT_TRAILER ,
    AnnotationCategories.MOBILE_PEDESTRIAN_CROSSING_SIGN ,

          ]
DYNAMIC = [
    AnnotationCategories.ARTICULATED_BUS ,
    AnnotationCategories.BICYCLE ,
    AnnotationCategories.ANIMAL ,
    AnnotationCategories.BICYCLIST ,
    AnnotationCategories.BOX_TRUCK ,
    AnnotationCategories.BUS ,
    AnnotationCategories.DOG ,
    AnnotationCategories.LARGE_VEHICLE ,
    AnnotationCategories.MESSAGE_BOARD_TRAILER ,
    AnnotationCategories.MOTORCYCLE ,
    AnnotationCategories.MOTORCYCLIST ,
    AnnotationCategories.OFFICIAL_SIGNALER ,
    AnnotationCategories.PEDESTRIAN ,
    AnnotationCategories.BICYCLIST ,
    AnnotationCategories.RAILED_VEHICLE ,
    AnnotationCategories.REGULAR_VEHICLE ,
    AnnotationCategories.SCHOOL_BUS ,
    AnnotationCategories.STROLLER ,
    AnnotationCategories.TRUCK ,
    AnnotationCategories.TRUCK_CAB ,
    AnnotationCategories.VEHICULAR_TRAILER ,
    AnnotationCategories.WHEELCHAIR ,
    AnnotationCategories.WHEELED_RIDER ,
    AnnotationCategories.WHEELED_DEVICE ,
          ]
NUM_CLASSES = len(STATIC) + len(DYNAMIC)



In [2]:
import os
os.chdir('/coc/pskynet2/apatni30/raster-net/comparison/cross_view_transformers/cross_view_transformer/')
print(os.getcwd())

/coc/pskynet2/apatni30/raster-net/comparison/cross_view_transformers/cross_view_transformer


In [3]:
from cross_view_transformer.data.argoverse2_dataset import Argoverse2Dataset

In [4]:
import pickle
with open('/srv/share2/apatni30/tmp/AV2_dataset.pkl', 'rb') as f:
    data = pickle.load(f)

In [5]:
pose = data.samples[0]['pose']
xpoints = np.arange(-data.bev_info['h_meters']/2,data.bev_info['h_meters']/2,data.bev_info['h_meters']/data.bev_info['h'])
ypoints = np.arange(-data.bev_info['w_meters']/2,data.bev_info['w_meters']/2,data.bev_info['w_meters']/data.bev_info['w'])
zpoints = np.array([i/10.0 for i in range(20)])
points_xy_wrt_src = np.vstack(map(np.ravel, np.meshgrid(xpoints, ypoints, zpoints))).T
points_xy_wrt_city = pose.transform_point_cloud(points_xy_wrt_src) 

/tmp/ipykernel_14197/1674624226.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  points_xy_wrt_src = np.vstack(map(np.ravel, np.meshgrid(xpoints, ypoints, zpoints))).T


In [6]:
cmap = np.zeros(shape=(points_xy_wrt_city.shape[0], 3))
drivable_area_raster = data.avm.raster_drivable_area_layer.get_raster_values_at_coords(points_xy_wrt_city, 0)
cmap[drivable_area_raster==0] = (0,0,0)
cmap[drivable_area_raster==1] = (220,220,220)

In [7]:
print(np.unique(drivable_area_raster, return_counts=True))
print(np.unique(cmap,return_counts=True))
points_xy_wrt_city

(array([0, 1]), array([613363, 186637]))
(array([  0., 220.]), array([1840089,  559911]))


array([[ 171.68438088, 3576.10885001,  -22.50234293],
       [ 171.68503285, 3576.10842759,  -22.40234594],
       [ 171.68568482, 3576.10800517,  -22.30234896],
       ...,
       [  68.80880637, 3672.08957022,  -19.72610128],
       [  68.80945834, 3672.08914781,  -19.6261043 ],
       [  68.81011031, 3672.08872539,  -19.52610732]])

In [8]:
import pdb

for annotation in CuboidList.from_feather(Path(data.log_specific_dataset_path, 'annotations.feather')):
    if annotation.timestamp_ns == data.samples[0]["token"]:
        annotation = annotation.transform(pose)      
        interior_points, is_interior = annotation.compute_interior_points(points_xy_wrt_city)          
        if annotation.category in STATIC:
            # print("static")
            # print(interior_points)
            # print(annotation.dst_SE3_object.translation)
            # print(annotation.width_m, annotation.length_m, annotation.height_m)
            # print(np.count_nonzero(is_interior))
            cmap[is_interior] = (255,127,80)            
        elif annotation.category in DYNAMIC:
            # print("dynamic")
            # print(interior_points)
            # print(annotation.dst_SE3_object.translation)
            # print(annotation.width_m, annotation.length_m, annotation.height_m)
            # print(np.count_nonzero(is_interior))
            cmap[is_interior] = (30,144,255)

In [9]:
print(np.unique(cmap,return_counts=True))

(array([  0.,  30.,  80., 127., 144., 220., 255.]), array([1836936,    8492,      20,      20,    8492,  537528,    8512]))


In [10]:
print(points_xy_wrt_city.shape)
print(data.bev_info['h'], data.bev_info['w'], 1)
print(data.bev_info['h_meters'], data.bev_info['w_meters'], 1)
print(cmap.shape)

(800000, 3)
200 200 1
100 100 1
(800000, 3)


In [11]:
output_bev = xyz_to_bev(
    xyz=points_xy_wrt_city,
    voxel_resolution=(data.bev_info['h'], data.bev_info['w'], 1),
    grid_size_m=(data.bev_info['h_meters'], data.bev_info['w_meters'], 1),
    cmap=cmap
)

ValueError: operands could not be broadcast together with shapes (800000,2) (3,) (800000,2) 